In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))

In [2]:
import cleanlab
import numpy as np

from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedKFold

from cleanlab.pruning import get_noise_indices

from pathlib import Path

import numpy as np
from numpy import random
import tensorflow as tf
from sklearn.base import BaseEstimator

import cv2

from tqdm import tqdm
from tensorflow.python.keras.preprocessing import dataset_utils

import utils
from model import DataCentricClassifier

tf.random.set_seed(123)

In [3]:
classifier = DataCentricClassifier(
    experiment_dir="../experiments/original/", 
    batch_size=8,
    random_seed=123,
    epoch=1
)
classifier.load_weights()

Creating ResNet50 model...


/home/omar/.cache/pypoetry/virtualenvs/dcc-8g-IaL7o-py3.8/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Loading paths from ../experiments/original/data/train
Found 2067 files belonging to 10 classes.
Loading paths from ../experiments/original/data/val
Found 813 files belonging to 10 classes.
Loading paths from ../experiments/original/data/test
Found 52 files belonging to 10 classes.
Loading weights from ../experiments/original/checkpoints/best_model


In [4]:
train_paths, train_labels = classifier.get_dataset_paths_and_labels("train")

In [5]:
train_size = len(train_paths)

In [6]:
X_train = np.arange(train_size)
y_train = train_labels

In [7]:
classifier.fit(X_train, y_train)

Filtered dataset size: 2067
Before shuffling: 
['../experiments/original/data/train/iii/b042daf0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ii/ab480bce-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aea96f9c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b075323e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac559310-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 1, 8, 2, 0]
After shuffling: 
['../experiments/original/data/train/vi/aaf25f62-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/abf4867e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac379130-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/abb3aeb0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aef8934c-ce5d-11eb-b317-38f9d35ea60f.png']
[5, 0, 0, 0, 8]
259/259 [==============================] - 7s 24ms/step - loss: 0.2499 - ac

In [8]:
def estimate_confident_joint_and_cv_pred_proba(
        X,
        s,
        clf,
        cv_n_folds=5,
        thresholds=None,
        seed=None,
        calibrate=True,
):
    cleanlab.latent_estimation.assert_inputs_are_valid(X, s)
    # Number of classes
    K = len(np.unique(s))

    # Ensure labels are of type np.array()
    s = np.asarray(s)

    # Create cross-validation object for out-of-sample predicted probabilities.
    # CV folds preserve the fraction of noisy positive and
    # noisy negative examples in each class.
    kf = StratifiedKFold(n_splits=cv_n_folds, shuffle=True, random_state=seed)

    # Intialize psx array
    psx = np.zeros((len(s), K))

    # Split X and s into "cv_n_folds" stratified folds.
    for k, (cv_train_idx, cv_holdout_idx) in enumerate(kf.split(X, s)):
        clf.load_weights()

        # Select the training and holdout cross-validated sets.
        X_train_cv, X_holdout_cv = X[cv_train_idx], X[cv_holdout_idx]
        s_train_cv, s_holdout_cv = s[cv_train_idx], s[cv_holdout_idx]

        # Fit the clf classifier to the training set and
        # predict on the holdout set and update psx.
        clf.fit(X_train_cv, s_train_cv)
        psx_cv = clf.predict_proba(X_holdout_cv, loader='train')  # P(s = k|x) # [:,1]
        psx[cv_holdout_idx] = psx_cv

    # Compute the confident counts, a K x K matrix for all pairs of labels.
    confident_joint = cleanlab.latent_estimation.compute_confident_joint(
        s=s,
        psx=psx,  # P(s = k|x)
        thresholds=thresholds,
        calibrate=calibrate,
    )

    return confident_joint, psx

In [9]:
cj, psx = estimate_confident_joint_and_cv_pred_proba(X_train, y_train, clf=classifier)

Loading weights from ../experiments/original/checkpoints/best_model
Filtered dataset size: 1653
Before shuffling: 
['../experiments/original/data/train/iii/b042daf0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ii/ab480bce-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aea96f9c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac559310-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b0735928-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 1, 8, 0, 2]
After shuffling: 
['../experiments/original/data/train/v/af31121c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae6dfac0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac49a5be-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae40e724-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac30e38a-ce5d-11eb-b317-38f9d35ea60f.png']
[4, 3, 0, 3, 0]
207/207

100%|██████████| 414/414 [00:13<00:00, 30.02it/s]


Loading weights from ../experiments/original/checkpoints/best_model
Filtered dataset size: 1653
Before shuffling: 
['../experiments/original/data/train/iii/b042daf0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ii/ab480bce-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b075323e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b0735928-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aede8826-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 1, 2, 2, 8]
After shuffling: 
['../experiments/original/data/train/iii/b0e38860-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae547924-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae1251f2-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/viii/ad0d5cca-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae5981f8-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 3, 3, 7, 3]

100%|██████████| 414/414 [00:15<00:00, 27.05it/s]


Loading weights from ../experiments/original/checkpoints/best_model
Filtered dataset size: 1654
Before shuffling: 
['../experiments/original/data/train/iii/b042daf0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ii/ab480bce-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aea96f9c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b075323e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac559310-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 1, 8, 2, 0]
After shuffling: 
['../experiments/original/data/train/ix/aea20d10-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aeb10b8a-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/vi/ab0cea1c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/vii/ac7a35c6-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/viii/ad77dad2-ce5d-11eb-b317-38f9d35ea60f.png']
[8, 8, 5, 6, 7]
2

100%|██████████| 413/413 [00:15<00:00, 26.74it/s]


Loading weights from ../experiments/original/checkpoints/best_model
Filtered dataset size: 1654
Before shuffling: 
['../experiments/original/data/train/ix/aea96f9c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b075323e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac559310-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b0735928-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/x/b01f03fa-ce5d-11eb-b317-38f9d35ea60f.png']
[8, 2, 0, 2, 9]
After shuffling: 
['../experiments/original/data/train/viii/ad7613f0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/x/b01bf9f8-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/abce0760-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b05ca9c6-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/vii/ac993098-ce5d-11eb-b317-38f9d35ea60f.png']
[7, 9, 0, 2, 6]
207

100%|██████████| 413/413 [00:12<00:00, 32.37it/s]


Loading weights from ../experiments/original/checkpoints/best_model
Filtered dataset size: 1654
Before shuffling: 
['../experiments/original/data/train/iii/b042daf0-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ii/ab480bce-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/ix/aea96f9c-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b075323e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/i/ac559310-ce5d-11eb-b317-38f9d35ea60f.png']
[2, 1, 8, 2, 0]
After shuffling: 
['../experiments/original/data/train/iv/ae0f025e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iii/b06d0c12-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/vii/ac7cda7e-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae3023f8-ce5d-11eb-b317-38f9d35ea60f.png',
 '../experiments/original/data/train/iv/ae8b5cdc-ce5d-11eb-b317-38f9d35ea60f.png']
[3, 2, 6, 3, 3]
20

100%|██████████| 413/413 [00:15<00:00, 26.12it/s]


In [10]:
cj, psx

(array([[250,   0,   8,   0,   0,   0,   0,   0,   0,   3],
        [  0, 157,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 185,   0,   0,   0,   1,   0,   0,   0],
        [  0,   0,   0, 281,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 196,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 179,   0,   0,   0,   2],
        [  0,   0,   0,   0,   0,   0, 193,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   1, 198,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 233,   1],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0, 179]]),
 array([[6.42085671e-02, 2.66878866e-03, 8.97208035e-01, ...,
         1.74721936e-04, 4.66833031e-03, 2.63664629e-02],
        [2.66794749e-02, 9.50848311e-02, 4.74436916e-02, ...,
         2.47119577e-03, 2.39000469e-03, 8.02319407e-01],
        [1.24065792e-07, 5.35884874e-06, 7.64636279e-05, ...,
         1.38415089e-05, 9.99685466e-01, 5.49185024e-05],
        ...,
        [1.22164050e-11, 

In [11]:
psx.shape

(2067, 10)

In [12]:
psx[0].round(2)

array([0.06, 0.  , 0.9 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.03])